# Hex and HexGrid

> Hexagonal rendering and grid management

In [ ]:
#| default_exp plot.hex

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import numpy as np
import math
from typing import List
from fastcore.basics import patch
from collections import namedtuple
from fasthtml.common import *
from HexMagic.plot.primitives import MapCord, MapSize, MapRect, MapPath, PrimitiveDemo
from HexMagic.styles import StyleCSS, SVGBuilder, SVGDef, Generatable


In [ ]:
#| export
from HexMagic.plot.cube import HexPosition

## Hex

In [ ]:
#| export
Edge = namedtuple('Edge', ['x1', 'y1', 'x2', 'y2'])


In [ ]:
#| export
class Hex:

     # Class-level constants for the 6 vertices (pointy-top orientation)
    _ANGLES = [(i * 60 - 30) * math.pi / 180 for i in range(6)]
    _COS = [math.cos(a) for a in _ANGLES]
    _SIN = [math.sin(a) for a in _ANGLES]

    radius: float
    center: MapCord
    style: StyleCSS

    _edgeLookup = [
                    (0, 1),   # E  -> vertices 0,1
                    (1, 2),   # SE -> vertices 1,2
                   (2, 3),   # SW -> vertices 2,3
                     (3, 4),   # W  -> vertices 3,4
                     (4, 5),   # NW -> vertices 4,5
                     (5, 0),   # NE -> vertices 5,0
    ]

    @classmethod
    def direction_to_edge(cls,x):
        
        return Hex._edgeLookup[x]
    

    _direction_to_vertices = [
            (2, 3),  # SW
            (3, 4),  # W
            (4, 5),  # NW
            (5, 0),  # NE
            (0, 1),  # E
            (1, 2),  # SE
        ]

    def edgeFrom(self, direction_idx: int) -> Edge:
        """Get the normalized edge tuple for a given direction index (matching HexPosition.directions())."""
        # Map from HexPosition.directions() order to vertex pairs
        # directions(): SW=0, W=1, NW=2, NE=3, E=4, SE=5
        
        
        i1, i2 = Hex._direction_to_vertices[direction_idx]
        v1 = self.v[i1]
        v2 = self.v[i2]
        
        x1, y1 = v1.x, v1.y
        x2, y2 = v2.x, v2.y
        
        if x1 < x2 or (x1 == x2 and y1 <= y2):
            return Edge(x1, y1, x2, y2)
        else:
            return Edge(x2, y2, x1, y1)



    def __init__(self,
        radius: float, #how far away are the vertexes
        center: MapCord, #where it is located
        style: StyleCSS = None, #how will it look
        label: str = "",
        labelStyle = "",
        v = None
        ):
        self.radius = radius
        self.center = MapCord(round(center.x,2),round(center.y,2))
        self.style = style
        self.label = label
        self.labelStyle = labelStyle
        
        width = self.radius  * 2
        self.bounds = MapSize(width,self.radius * 2 )

        #cache the vertices
        # Cache vertices using precomputed trig
        if v is None:
            self.v = [
                MapCord(
                    int(self.center.x + self.radius * Hex._COS[i]),
                    int(self.center.y + self.radius * Hex._SIN[i])
                )
                for i in range(6)
            ]
        else:
            self.v = v
      
    
    def vertices(self) -> List[MapCord]:
        """Get the vertices of a hexagon centered at (center_x, center_y)."""
        return self.v
    
    def contains(self, point: MapCord) -> bool:
        """Check if a point is inside a hexagon."""
        distance = math.sqrt((point.x - self.center.x)**2 + (point.y - self.center.y)**2)
        return distance <= self.radius * 0.9
    
    def svg(self,attrs=None) -> str:
        ret = "<polygon points=\""
        for point in self.vertices():
            ret += f"{point.x:.0f},{point.y:.0f} "
        ret += f"\" class=\"{self.style.name}\""
        if attrs is not None:
            for key, value in attrs.items():
                ret += f" {key}='{value}'"

        ret += "/>"
        return ret

    def angle(self, other: 'Hex') -> float:
        """Calculate angle from this hex to another hex"""
        return math.atan2(other.center.y - self.center.y, other.center.x - self.center.x)

    def right(self):
        center = MapCord(self.center.x + self.bounds.width, self.center.y)
        return Hex(self.radius,center,self.style)


In [ ]:
#| export

@patch
def edges(self: Hex) -> set[Edge]:
    """Return the edges of the hexagon as a set of normalized tuples (x1, y1, x2, y2).
    Each edge is normalized so that x1 < x2, or if x1 == x2 then y1 <= y2.
    Coordinates are converted to ints for set operations."""
    edges = set()
    vertices = self.vertices()
    
    for pos in range(6):
        edge = self.edgeFrom(pos)
        edges.add(edge)
    
    return edges


Lets make a named tuple for edges

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good idea! A named tuple will make your edge code much more readable. You could define it like this:

```python
Edge = namedtuple('Edge', ['x1', 'y1', 'x2', 'y2'])
```

You've already imported `namedtuple` from `collections`, so you're all set. 

Once you add this, you'll want to update `edgeFrom()` and `edges()` to return `Edge` instances instead of plain tuples. Want to tackle that yourself, or would you like me to show you what the changes would look like?

In [ ]:

sampleHex = Hex(radius=50,center=MapCord(100,100))
sampleHex.vertices(), "\n-- We round edges so we can compare --\n", sampleHex.edges()

([(143,75), (143,125), (100,150), (56,125), (56,75), (99,50)],
 '\n-- We round edges so we can compare --\n',
 {Edge(x1=56, y1=75, x2=56, y2=125),
  Edge(x1=56, y1=75, x2=99, y2=50),
  Edge(x1=56, y1=125, x2=100, y2=150),
  Edge(x1=99, y1=50, x2=143, y2=75),
  Edge(x1=100, y1=150, x2=143, y2=125),
  Edge(x1=143, y1=75, x2=143, y2=125)})

In [ ]:
hexStyle = StyleCSS("HexStyle",fill="yellow",stroke="blue",stroke_width=2)
canvas = SVGBuilder()
canvas.width=200 ;canvas.height=200
canvas.add_style(hexStyle)

#add our hex to the canvas
sampleHex = Hex(radius=50,center=MapCord(100,100),style=hexStyle)
canvas.adjust("main",sampleHex.svg())

#show our work
canvas.show()
canvas.adjust("main",sampleHex.svg())

#show our work
canvas.show()

### Hex Drawing

In [ ]:
#| export
def hexSVG(size:MapSize,levels = 2, pad=5, fill="#f4edb2ff",eles=StyleCSS.elevations()):
    myStyles = []
    radius = size.width/2
    center = MapCord(radius,size.height/2)
    
    canvas = ""
    hexRadius = radius
    polySize = MapSize(size.width,size.height)
    
    for i in range(levels):
        aStyle = eles[i]
        myStyles.append(aStyle)
        aHex = Hex(radius=hexRadius-4,center=center,style=aStyle)
        debug = aHex.verticalStretch(polySize)
        #print(f"round {i},{aHex.center} radius{aHex.radius},siz:{polySize}",debug)
        canvas += debug
        polySize = MapSize(polySize.width-pad*2,polySize.height-pad*2)
        hexRadius -= pad
        canvas += "\n"

    aStyle = StyleCSS("Snow",fill=fill)
    
    myStyles.append(aStyle)
    aHex = Hex(radius=hexRadius-4,center=center,style=aStyle)
    canvas += aHex.verticalStretch(polySize)
    
    sheet  = Style("\n ".join([x.__str__() for x in myStyles]) )
    #hSVG =  embedSVG(sheet + canvas,size=size)
    retRender = SVGBuilder()
    retRender.width = size.width
    retRender.height = size.height
    for x in myStyles:
        retRender.add_style(x)

    retRender.updateLayers([canvas])
    return Div(NotStr(retRender.xml()),style="""
                width: 100%; 
                height: 100%; 
                position: absolute; 
                filter: drop-shadow(4px 4px 6px rgba(0,0,0,0.3));
                transition: all 0.2s ease;
             display: flex;
            justify-content: center;
                """)
    



def hexBackground( content=None,levels = 2, pad=5,fill="#f4edb2ff",size=MapSize(300,300)):
     return Div(
        hexSVG(size,levels,pad,fill),
   Div(content, style="position: relative; z-index: 1;"),
        style=f"""
            position: relative; 
            width: {size.width}px; 
            height: {size.height}px;
            transition: all 0.2s ease;
            display: flex;
            justify-content: center;
            align-items: center;
        """
     )

In [ ]:
#| export
@patch
def hexIcon(self: SVGBuilder, size: int) -> str:
    """Generate hex-shaped icon cropped from center of SVG."""
    cx, cy = self.width / 2, self.height / 2
    
    # Find largest hex that fits in original image
    # For pointy-top: width = r*√3, height = r*2
    # So: r = min(width/√3, height/2)
    max_radius = min(self.width / math.sqrt(3), self.height / 2)
    
    # Hex bounding box in original coordinates
    hex_width = max_radius * math.sqrt(3)
    hex_height = max_radius * 2
    vb_x = cx - hex_width / 2
    vb_y = cy - hex_height / 2
    
    # Clip path hex - coordinates in original space
    aHex = Hex(radius=max_radius, center=MapCord(cx, cy), style=StyleCSS("hex"))
    clipDef = SVGDef("clipPath", "hex-clip", aHex.verticalStretch(size=MapSize(hex_width, hex_height)))  # just the polygon points
    self.add_definition(clipDef)
    
    # Output maintains hex aspect ratio
    out_width = size * math.sqrt(3) / 2  # ~0.866 * size
    out_height = size
    
    viewBox = f"{vb_x} {vb_y} {hex_width} {hex_height}"
    
    return f'''<?xml version='1.0' encoding='utf-8'?>
<svg xmlns="http://www.w3.org/2000/svg" 
     width="{out_width:.1f}" 
     height="{out_height}"
     viewBox="{viewBox}">
{self._header()}
<g clip-path="url(#hex-clip)">
{self.body}
</g>
</svg>'''

## Callback

In [ ]:
#| export
class HexWrapper:
    """This class lets us have a call back route if a hex is clicked. """
    def __init__(self,
    header:str = "", # goes in front of the body 
    footer: str = "", # goes in the back
    callBack = lambda grid,index: None #how individual cells are called back
    ):
        self.header = ""
        self.footer = ""
        self.callBack = callBack

    def route(path:str= "/hex_clicked",target:str="#map"):
        return lambda grid,index :  {
        "hx-post": f"{path}", #The post gives the route back to the url
        "hx-vals": f'{{"hex_id":{index}}}', # the parameter needs to be called the same in the route
        "hx-target": f"{target}" # the element we need to update
    }

## HexGrid

In [ ]:
#| export
class HexGrid:
    """Hexagonal grid with cube coordinate support."""

    _SQRT3 = math.sqrt(3) 

    def __init__(self, 
                 nRows: int,
                 nCols: int, 
                 radius: float,
                 style: StyleCSS,
                 offset: MapCord = None):
        self.nRows = nRows
        self.nCols = nCols
        self.radius = radius
        self.style = style
        self.offset = offset or MapCord(0, 0)
        
        self.builder = SVGBuilder()
        self.builder.add_style(style)
        for x in StyleCSS.elevations():
            self.builder.add_style(x)
        
        self._build_hexes()

    @classmethod
    def from_bounds(cls, bounds: MapRect, radius: float=25, style: StyleCSS=StyleCSS("Hex")) -> 'HexGrid':
        """Create grid to fill a bounding rectangle."""

        nRows = int(bounds.dimensons.height / radius)
        nCols = int(bounds.dimensons.width / radius)
        offset = MapCord(bounds.origin.x - radius, bounds.origin.y - radius)
        
        return cls(nRows, nCols, radius, style, offset)

    
    

    
    @classmethod
    def centered(cls, rings: int, radius: float, style: StyleCSS, 
                 center: MapCord = None) -> 'HexGrid':
        """Create grid with specified rings around a center point."""
        n = 2 * rings + 1
        
        # Create grid first with no offset
        grid = cls(nRows=n, nCols=n, radius=radius, style=style, offset=MapCord(0, 0))
        
        if center is not None:
            # Calculate offset to place middle hex at center
            natural_middle = grid._middle_hex_natural_position()
            grid.offset = MapCord(
                center.x - natural_middle.x,
                center.y - natural_middle.y
            )
            grid._build_hexes()
        
        return grid
    
    @property
    def middle(self) -> int:
        """Middle hex index."""
        return (self.nRows // 2) * self.nCols + (self.nCols // 2)
    
    @property
    def bounds(self) -> MapRect:
        """Bounding rectangle of grid."""
        if not self.hexes:
            return MapRect(MapCord(0, 0), MapSize(0, 0))
        return MapRect(
            self.hexes[0].center,
            MapSize(self.nCols * self.radius * HexGrid._SQRT3, 
                    self.nRows * self.radius * 1.5)
        )


    def text(self, cb=lambda s, i: i):
        i = 0
        for row in range(self.nRows):
            line = "|"
            for col in range(self.nCols):
                line += f" {cb(self, i):5}"
                i += 1
            print(line + " |")

    def rowPartity(self, index):
        "This returns whether a row is even or odd"
        return int(index / self.nCols) % 2

    @property
    def midpoint(self):
        return int(len(self.hexes)/2)



In [ ]:
#| export
@patch
def _build_hexes(self:HexGrid):
    """Build hex array using vectorized numpy operations."""
    # Create row and col indices
    rows = np.arange(self.nRows)
    cols = np.arange(self.nCols)
    row_grid, col_grid = np.meshgrid(rows, cols, indexing='ij')
    
    # Flatten to 1D arrays
    row_flat = row_grid.flatten()
    col_flat = col_grid.flatten()
    
    # Calculate all centers at once
    width = HexGrid._SQRT3 * self.radius
    height = 2 * self.radius
    
    x_centers = width * (col_flat + 0.5 * (row_flat % 2)) + self.offset.x + self.radius
    y_centers = height * 0.75 * row_flat + self.offset.y + self.radius
    
    # Calculate all vertices at once using broadcasting
    # Shape: (n_hexes, 6) for each coordinate
    angles = np.array(Hex._ANGLES)  # (6,)
    cos_vals = np.array(Hex._COS)   # (6,)
    sin_vals = np.array(Hex._SIN)   # (6,)
    
    # Broadcast to (n_hexes, 6)
    x_vertices = x_centers[:, np.newaxis] + self.radius * cos_vals[np.newaxis, :]
    y_vertices = y_centers[:, np.newaxis] + self.radius * sin_vals[np.newaxis, :]
    
    # Round to integers
    x_vertices = np.round(x_vertices).astype(int)
    y_vertices = np.round(y_vertices).astype(int)
    
    # Create Hex objects with pre-computed vertices
    self.hexes = []
    for i in range(len(row_flat)):
        center = MapCord(round(x_centers[i], 2), round(y_centers[i], 2))
        vertices = [MapCord(x_vertices[i, j], y_vertices[i, j]) for j in range(6)]
        self.hexes.append(Hex(self.radius, center, self.style, v=vertices))
    
    self._update_builder_size()

@patch
def _hex_at(self:HexGrid, row: int, col: int, style: StyleCSS) -> Hex:
    """Calculate hex at grid position."""
    width = HexGrid._SQRT3 * self.radius
    height = 2 * self.radius
    
    # Add radius to offset so first hex center is actually inside the bounds
    x = width * (col + 0.5 * (row % 2)) + self.offset.x + self.radius
    y = height * 0.75 * row + self.offset.y + self.radius
    
    return Hex(self.radius, MapCord(x, y), style)


@patch
def _update_builder_size(self:HexGrid):
    """Update SVG builder dimensions."""
    if self.hexes:
        self.builder.width = max(h.center.x for h in self.hexes) + self.radius
        self.builder.height = max(h.center.y for h in self.hexes) + self.radius
@patch
def _middle_hex_natural_position(self:HexGrid) -> MapCord:
    """Where the middle hex center would be with zero offset."""
    middle_row = self.nRows // 2
    middle_col = self.nCols // 2
    
    width = HexGrid._SQRT3 * self.radius
    height = 2 * self.radius
    
    x = width * (middle_col + 0.5 * (middle_row % 2))
    y = height * 0.75 * middle_row
    
    return MapCord(x, y)


In [ ]:
#| export

@patch
def adjustRadius(self:HexGrid, new_radius: float,keepMiddle=False):
    """Adjust radius while keeping middle hex at same pixel position."""
    if not self.hexes:
        self.radius = new_radius
        return
    
    # Get current middle hex pixel position
    middle_idx = self.middle
    old_middle_pos = self.hexes[middle_idx].center
    
    # Update radius
    self.radius = new_radius
    
    # Calculate where middle would be with zero offset
    natural_middle = self._middle_hex_natural_position()
    
    if keepMiddle:
        # Offset to keep middle at same pixel position
        self.offset = MapCord(
            old_middle_pos.x - natural_middle.x + self.offset.x,
            old_middle_pos.y - natural_middle.y + self.offset.y
        )
    else:
        self.offset = MapCord(-self.radius,-self.radius)
    
    # Rebuild hexes with new radius and offset
    self._build_hexes()

### Conversions

In [ ]:
#| export
@patch
def index_to_hexposition(self: HexGrid, index: int, origin_index: int = None) -> HexPosition:
    """Convert grid index to HexPosition relative to origin_index."""

    if origin_index is None:
        origin_index = int(len(self.hexes)/2)
        
    # Convert both indices to row/col
    row, col = self.index_to_row_col(index)
    origin_row, origin_col = self.index_to_row_col(origin_index)
    
    # Convert to cube coordinates (odd-r offset)
    q = col - (row - (row & 1)) // 2
    r = row
    s = -q - r
    
    origin_q = origin_col - (origin_row - (origin_row & 1)) // 2
    origin_r = origin_row
    origin_s = -origin_q - origin_r
    
    # Return relative position
    return HexPosition(q - origin_q, r - origin_r, s - origin_s)

@patch
def hexposition_to_index(self: HexGrid, hexpos: HexPosition, origin_index:int = None) -> int:
    if origin_index is None:
        origin_index = int(len(self.hexes)/2)
    """Convert HexPosition (relative to origin) back to grid index.
    Returns -1 if out of bounds."""
    # Get origin's cube coordinates
    origin_row, origin_col = self.index_to_row_col(origin_index)
    origin_q = origin_col - (origin_row - (origin_row & 1)) // 2
    origin_r = origin_row
    origin_s = -origin_q - origin_r
    
    # Add relative position to origin
    abs_q = hexpos.q + origin_q
    abs_r = hexpos.r + origin_r
    abs_s = hexpos.s + origin_s
    
    # Convert cube back to odd-r offset
    row = abs_r
    col = abs_q + (abs_r - (abs_r & 1)) // 2
    
    # Convert to index and check bounds
    return self.row_col_to_index(row, col)


In [ ]:
#| export
HexGrid.i2hp = index_to_hexposition
HexGrid.hp2i = hexposition_to_index

In [ ]:
#| export
@patch
def index_to_row_col(self: HexGrid, index: int) -> tuple[int, int]:
    """Convert flat grid index to (row, col)."""
    row = index // self.nCols
    col = index % self.nCols
    return row, col

@patch
def row_col_to_index(self: HexGrid, row: int, col: int) -> int:
    """Convert (row, col) to flat grid index. Returns -1 if out of bounds."""
    if row < 0 or row >= self.nRows or col < 0 or col >= self.nCols:
        return -1
    return row * self.nCols + col

In [ ]:
#| export
@patch
def neighborsOf(self: HexGrid, index: int,ring=1) -> list[int]:
    """Get all valid neighbor indices using HexPosition."""
    ring_hexpositions = HexPosition.origin().ring(ring)
    neighbor_indices = [self.hexposition_to_index(hp, index) for hp in ring_hexpositions]
    return [i for i in neighbor_indices if i >= 0]  # Filter out-of-bounds


#### Boundries
These functions help when we need outline hexes

In [ ]:
#| export
@patch
def direction_index(self: HexPosition) -> int | None:
    """Return direction index (0-5) if this is a unit direction, else None."""
    directions = HexPosition.directions()
    for idx, d in enumerate(directions):
        if d == self:
            return idx
    return None

@patch
def commonEdge(self: HexGrid, i: int, j: int) -> tuple[int, int, int, int] | None:
    """Get the shared edge between two hex indices, or None if not adjacent."""
    pos = self.index_to_hexposition(j, i)
    dir_idx = pos.direction_index()
    if dir_idx is None:
        return None
    return self.hexes[i].edgeFrom(dir_idx)



## Drawing

In [ ]:
#| export
@patch
def legendOverlayHorizontal(self: SVGBuilder,
    styles: [StyleCSS],
    bg_fill="white",
    xOffset=10,
    yOffset=10,
    max_width=None,  # defaults to self.width - xOffset
    char_width=7,    # approximate width per character
    radius=12,
    pad=8
    ):
    """Horizontal legend that wraps to multiple rows if needed."""
    
    if max_width is None:
        max_width = self.width - xOffset * 2
    
    ret = ""
    row_height = radius * 2 + pad
    
    # First pass: calculate positions and total height
    positions = []
    x = radius + pad
    y = radius + pad
    
    for style in styles:
        name = " ".join(style.name.split("_"))
        text_width = len(name) * char_width
        item_width = radius * 2 + pad + text_width + pad * 2
        
        # Check if we need to wrap
        if x + item_width > max_width and x > radius + pad:
            x = radius + pad
            y += row_height
        
        positions.append((x, y, style, name))
        x += item_width
    
    # Calculate total dimensions
    total_height = y + radius + pad
    total_width = max_width
    
    # Add background rectangle
    if bg_fill:
        ret += f'\t<rect x="{xOffset}" y="{yOffset}" width="{total_width}" height="{total_height}" fill="{bg_fill}" rx="5"/>\n'
    
    # Draw items
    for x, y, style, name in positions:
        cx = xOffset + x
        cy = yOffset + y
        ret += f'\t<circle cx="{cx}" cy="{cy}" r="{radius}" class="{style.name}"/>\n'
        ret += f'\t<text x="{cx + radius + pad}" y="{cy}" text-anchor="start" dy="0.35em" class="keyLabel">{name}</text>\n'
    
    return ret


In [ ]:
#| export
@patch
def styledHexes(self:HexGrid,wrapper:HexWrapper = HexWrapper()):

        testBody = ""
        hexWrap = wrapper.callBack

        exportSize = MapSize(max([hex.center.x for hex in self.hexes]),
        max([hex.center.y for hex in self.hexes]))
        self.builder.width = exportSize.width
        self.builder.height = exportSize.height

        for i, hex in enumerate(self.hexes):
            testBody += "\t" +  hex.svg(hexWrap(self,i)) + "\n"
            if len(hex.label) > 0:
                testBody += f"\t\t<text x=\"{hex.center.x}\" y=\"{hex.center.y}\" text-anchor=\"middle\" dominant-baseline=\"middle\""
                if len(hex.labelStyle) > 1:
                    testBody += f" class=\"{hex.labelStyle}\""
                
                testBody += f">{hex.label}</text>\n"
        
        return testBody

@patch
def update(self:HexGrid,wrapper:HexWrapper = HexWrapper(),layer_name="hexes"):

        testBody = self.styledHexes(wrapper=wrapper)
        self.builder.adjust(layer_name,testBody)



In [ ]:
#| export
@patch
def sampleGrid(self:PrimitiveDemo,hexDim = 2, fill = "white",makeLabels = False):

    #drawing set up
    hexStyle = StyleCSS("HexStyle",fill=fill,stroke="blue",stroke_width=2)
    labelStyle = StyleCSS("labelStyle",fill=fill,stroke="black",stroke_width=1)
    
    grid = HexGrid.centered(hexDim,radius=40,style=hexStyle)

    #Need to add styles as we go along
    grid.builder.add_style(labelStyle)

    #itterate through the hexes
    for i in range(len(grid.hexes)):
        if makeLabels:
            grid.hexes[i].label = str(i) 
        grid.hexes[i].labelStyle = labelStyle.name

    grid.update()
    return grid
    


In [ ]:

grid = PrimitiveDemo().sampleGrid(makeLabels=True)
grid.builder.show()

In [ ]:
#| export
@patch
def arrow(self: HexGrid, start:int, end:int, style = StyleCSS("arrow", stroke="black",stroke_width=1),fromMiddle=False,factor=0.25) -> str:
    self.builder.add_style(style)
    
    

        # Get the hex positions. This defaults to postion 0
    start_pos = self.index_to_hexposition(start)
    end_pos = self.index_to_hexposition(end)
    
    # Draw a line between them using cube coordinate interpolation
    path_positions = [self.hexposition_to_index(x) for x in start_pos.line_to(end_pos)]
    start_center = self.hexes[path_positions[0]].center
    end_center = self.hexes[path_positions[1]].center

    
    # Calculate point at 25% along the way
    start_x = start_center.x + factor * (end_center.x - start_center.x)
    start_y = start_center.y + factor * (end_center.y - start_center.y)
    points = [MapCord(start_x, start_y)]
    for x in path_positions[1:-1]:
        points.append(self.hexes[x].center)
    
    # Calculate point at 75% along the way
    start_center = self.hexes[path_positions[-2]].center
    end_center = self.hexes[path_positions[-1]].center
    end_x = start_center.x + (1 - factor) * (end_center.x - start_center.x)
    end_y = start_center.y + (1 - factor) * (end_center.y - start_center.y)
    points.append(MapCord(end_x, end_y))
    
    path = MapPath(points, style)
    return path.with_arrowhead()

NameError: name 'patch' is not defined

In [ ]:
grid = PrimitiveDemo().sampleGrid()

#mark the treasure
grid.hexes[grid.midpoint].label = "X"

#lets have a directions layer
arrowLayer = ""

#Use the realtive position class called HexPosition
for position in HexPosition.directions():
    i = grid.hexposition_to_index(position, 12)
    grid.hexes[i].label = position.label
    arrowLayer += grid.arrow(12,i)

grid.update()

#we can add a layer or update one in a builder using the adjust method
grid.builder.adjust("arrows",arrowLayer)

grid.builder.show()

### Gradients

In [ ]:
#| export
class LinearGradient(Generatable):

    def __init__(self,grid:HexGrid,startHex:int,endHex:int,startColor:str,endColor:str,):
        self.startHex = startHex
        self.endHex = endHex
        self.startColor = startColor
        self.endColor = endColor
        self.grid = grid
        self.name =  f"grad_{startHex}_{endHex}"

    
    def poly(self):
        startH = self.grid.hexes[self.startHex]
        endH = self.grid.hexes[self.endHex]
        testBody = ""

        edges = list(startH.edges().intersection(endH.edges()))
        if len(edges) > 0:
            edge = edges[0]

            testBody += "\t" +  "<polygon points=\""
            testBody += f"{startH.center.x:.0f},{startH.center.y:.0f} "
            testBody += f"{edge[0]},{edge[1]} "
            testBody += f"{endH.center.x:.0f},{endH.center.y:.0f} "
            testBody += f"{edge[2]},{edge[3]} "
            testBody += f"\" fill=\"url(#{self.name})\" />\n"

        return testBody

    def generate(self) -> str:
        startH = self.grid.hexes[self.startHex]
        endH = self.grid.hexes[self.endHex]
        
        # Calculate vector from start to end
        dx = endH.center.x - startH.center.x
        dy = endH.center.y - startH.center.y
        
        # Use actual coordinates for the gradient
        svg = f'<linearGradient id="{self.name}" '
        svg += f'x1="{startH.center.x}" y1="{startH.center.y}" '
        svg += f'x2="{endH.center.x}" y2="{endH.center.y}" '
        svg += 'gradientUnits="userSpaceOnUse">\n'
        svg += f'  <stop offset="0%" stop-color="{self.startColor}" />\n'
        svg += f'  <stop offset="100%" stop-color="{self.endColor}" />\n'
        svg += '</linearGradient>\n'
        return svg

In [ ]:
#| export
@patch
def radial_gradient(self: HexGrid, lookup: dict= {5:"#007fff",6:"#07ff66ff",10:"#ff005dff]"} ):
    """Use overlapping radial gradients for smoother blending."""
    testBody = ""
    
    for i, color in lookup.items():
        hex = self.hexes[i]
        grad_id = f"radial_{i}"
        
        # Create radial gradient fading to transparent
        grad = f'''<radialGradient id="{grad_id}" cx="{hex.center.x}" cy="{hex.center.y}" 
                    r="{self.radius * 1.5}" gradientUnits="userSpaceOnUse">
            <stop offset="0%" stop-color="{color}" stop-opacity="1"/>
            <stop offset="70%" stop-color="{color}" stop-opacity="0.5"/>
            <stop offset="100%" stop-color="{color}" stop-opacity="0"/>
        </radialGradient>'''
        self.builder.add_definition(SVGDef("", grad_id, grad, raw=True))
        
        # Draw a circle with this gradient
        testBody += f'<circle cx="{hex.center.x}" cy="{hex.center.y}" r="{self.radius * 1.5}" fill="url(#{grad_id})"/>\n'
    
    return testBody

In [ ]:
#| export
@patch
def gradient(self:HexGrid,lookup = {5:"#007fff",6:"#07ff66ff",10:"#ff005dff]"} ):
    testBody = ""
   
    exportSize = MapSize(max([hex.center.x for hex in self.hexes]),
    max([hex.center.y for hex in self.hexes]))
    self.builder.width = exportSize.width
    self.builder.height = exportSize.height
    

    for i ,v in lookup.items():
        hex = self.hexes[i]

        neighs = [j for j in self.neighborsOf(i) if j > i]
        for j in neighs:
            if j in lookup:
                lg = LinearGradient(self,i,j,lookup[i],lookup[j])
                self.builder.add_definition(lg)
                testBody += lg.poly()

    return testBody


I was thinking about how to add a legend below our hexgrid in a standarized way. right now we can easily put it over which would block part of the graph so we want to make sure that its builder creates the proper amount of height for both the grid and the legend which could have multiple rows

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great idea! Currently `legendOverlayHorizontal` overlays at a fixed position, but you want it *below* the grid with the builder's height automatically adjusted.

Here's the key insight: you need to know the legend's height *before* you can set the builder's total height. So the approach could be:

1. **Calculate legend height first** — do a "dry run" to figure out how many rows the legend needs
2. **Position legend below grid** — set `yOffset` to `grid.builder.height + some_padding`
3. **Expand builder height** — add the legend height to `grid.builder.height`

One way to structure this: split the current method into two parts:
- A helper that calculates positions and returns the total legend height
- The rendering part that uses those positions

Does that approach make sense? Would you like to start by extracting the height calculation logic from `legendOverlayHorizontal`, or would you prefer a different structure?

Can you split the legend into two parts (the size calc) and the drawing/layout. As a bonus have it be able to use hexagons as well as circles for the legend items. We have great hex code.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Here's the refactored legend with separate layout calculation and rendering:

```python
@patch
def legendLayout(self: SVGBuilder,
    styles: list[StyleCSS],
    max_width: float = None,
    char_width: float = 7,
    radius: float = 12,
    pad: float = 8
) -> tuple[list[tuple], float, float]:
    """Calculate legend layout without drawing. Returns (positions, total_width, total_height).
    Each position is (x, y, style, label_text)."""
    
    if max_width is None:
        max_width = self.width
    
    row_height = radius * 2 + pad
    positions = []
    x = radius + pad
    y = radius + pad
    
    for style in styles:
        name = " ".join(style.name.split("_"))
        text_width = len(name) * char_width
        item_width = radius * 2 + pad + text_width + pad * 2
        
        # Wrap to next row if needed
        if x + item_width > max_width and x > radius + pad:
            x = radius + pad
            y += row_height
        
        positions.append((x, y, style, name))
        x += item_width
    
    total_height = y + radius + pad
    return positions, max_width, total_height


@patch
def legendRender(self: SVGBuilder,
    positions: list[tuple],
    width: float,
    height: float,
    x_offset: float = 0,
    y_offset: float = 0,
    bg_fill: str = "white",
    radius: float = 12,
    pad: float = 8,
    use_hex: bool = False
) -> str:
    """Render legend from pre-calculated positions."""
    
    ret = ""
    
    # Background
    if bg_fill:
        ret += f'\t<rect x="{x_offset}" y="{y_offset}" width="{width}" height="{height}" fill="{bg_fill}" rx="5"/>\n'
    
    # Items
    for x, y, style, name in positions:
        cx = x_offset + x
        cy = y_offset + y
        
        if use_hex:
            # Use a small hex
            hex_item = Hex(radius=radius, center=MapCord(cx, cy), style=style)
            ret += f'\t{hex_item.svg()}\n'
        else:
            ret += f'\t<circle cx="{cx}" cy="{cy}" r="{radius}" class="{style.name}"/>\n'
        
        ret += f'\t<text x="{cx + radius + pad}" y="{cy}" text-anchor="start" dy="0.35em" class="keyLabel">{name}</text>\n'
    
    return ret


@patch
def legendBelow(self: SVGBuilder,
    styles: list[StyleCSS],
    gap: float = 10,
    bg_fill: str = "white",
    char_width: float = 7,
    radius: float = 12,
    pad: float = 8,
    use_hex: bool = False
) -> str:
    """Add legend below current content, expanding builder height as needed."""
    
    # Calculate layout
    positions, legend_width, legend_height = self.legendLayout(
        styles, max_width=self.width, char_width=char_width, radius=radius, pad=pad
    )
    
    # Position below current content
    y_offset = self.height + gap
    
    # Expand builder height
    self.height = y_offset + legend_height
    
    # Render
    return self.legendRender(
        positions, legend_width, legend_height,
        x_offset=0, y_offset=y_offset,
        bg_fill=bg_fill, radius=radius, pad=pad, use_hex=use_hex
    )
```

Usage would look like:

```python
# Add legend below and auto-expand height
legend_svg = grid.builder.legendBelow(my_styles, use_hex=True)
grid.builder.adjust("legend", legend_svg)
```

The split gives you flexibility — you can also use `legendLayout` + `legendRender` separately if you need custom positioning.

In [ ]:
#| export
@patch
def legendLayout(self: SVGBuilder,
    styles: list[StyleCSS],
    max_width: float = None,
    char_width: float = 7,
    radius: float = 12,
    pad: float = 8
) -> tuple[list[tuple], float, float]:
    """Calculate legend layout without drawing. Returns (positions, total_width, total_height).
    Each position is (x, y, style, label_text)."""
    
    if max_width is None:
        max_width = self.width
    
    row_height = radius * 2 + pad
    positions = []
    x = radius + pad
    y = radius + pad
    
    for style in styles:
        name = " ".join(style.name.split("_"))
        text_width = len(name) * char_width
        item_width = radius * 2 + pad + text_width + pad * 2
        
        # Wrap to next row if needed
        if x + item_width > max_width and x > radius + pad:
            x = radius + pad
            y += row_height
        
        positions.append((x, y, style, name))
        x += item_width
    
    total_height = y + radius + pad
    return positions, max_width, total_height


@patch
def legendRender(self: SVGBuilder,
    positions: list[tuple],
    width: float,
    height: float,
    x_offset: float = 0,
    y_offset: float = 0,
    bg_fill: str = "white",
    radius: float = 12,
    pad: float = 8,
    use_hex: bool = False
) -> str:
    """Render legend from pre-calculated positions."""
    
    ret = ""
    
    # Background
    if bg_fill:
        ret += f'\t<rect x="{x_offset}" y="{y_offset}" width="{width}" height="{height}" fill="{bg_fill}" rx="5"/>\n'
    
    # Items
    for x, y, style, name in positions:
        cx = x_offset + x
        cy = y_offset + y
        
        if use_hex:
            # Use a small hex
            hex_item = Hex(radius=radius, center=MapCord(cx, cy), style=style)
            ret += f'\t{hex_item.svg()}\n'
        else:
            ret += f'\t<circle cx="{cx}" cy="{cy}" r="{radius}" class="{style.name}"/>\n'
        
        ret += f'\t<text x="{cx + radius + pad}" y="{cy}" text-anchor="start" dy="0.35em" class="keyLabel">{name}</text>\n'
    
    return ret


@patch
def legendBelow(self: SVGBuilder,
    styles: list[StyleCSS],
    gap: float = 10,
    bg_fill: str = "white",
    char_width: float = 7,
    radius: float = 12,
    pad: float = 8,
    use_hex: bool = False
) -> str:
    """Add legend below current content, expanding builder height as needed."""
    
    # Calculate layout
    positions, legend_width, legend_height = self.legendLayout(
        styles, max_width=self.width, char_width=char_width, radius=radius, pad=pad
    )
    
    # Position below current content
    y_offset = self.height + gap
    
    # Expand builder height
    self.height = y_offset + legend_height
    
    # Render
    return self.legendRender(
        positions, legend_width, legend_height,
        x_offset=0, y_offset=y_offset,
        bg_fill=bg_fill, radius=radius, pad=pad, use_hex=use_hex
    )


I was wondering if we ran the legend a second time we would cause the builder to grow when it didn't need to

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good catch! Each call to `legendBelow` adds `gap + legend_height` to the builder height, so repeated calls would keep growing it.

A few ways to fix this:

**Option 1: Track original content height**
```python
@patch
def legendBelow(self: SVGBuilder,
    styles: list[StyleCSS],
    gap: float = 10,
    bg_fill: str = "white",
    char_width: float = 7,
    radius: float = 12,
    pad: float = 8,
    use_hex: bool = False
) -> str:
    """Add legend below current content, expanding builder height as needed."""
    
    # Use stored content height if we've rendered a legend before, else current height
    content_height = getattr(self, '_content_height', self.height)
    self._content_height = content_height  # Remember for next time
    
    # Calculate layout
    positions, legend_width, legend_height = self.legendLayout(
        styles, max_width=self.width, char_width=char_width, radius=radius, pad=pad
    )
    
    # Position below content (not below previous legend)
    y_offset = content_height + gap
    
    # Set total height
    self.height = y_offset + legend_height
    
    # Render
    return self.legendRender(
        positions, legend_width, legend_height,
        x_offset=0, y_offset=y_offset,
        bg_fill=bg_fill, radius=radius, pad=pad, use_hex=use_hex
    )
```

**Option 2: Reset method**
Add a `clearLegend()` that restores `_content_height` if you want to remove the legend space.

Option 1 is probably cleanest — it's idempotent and "just works" on repeated calls.